In [0]:
from google.colab import drive
GOOGLE_DRIVE_MOUNT = "/content/gdrive"
drive.mount(GOOGLE_DRIVE_MOUNT)

Mounted at /content/gdrive


In [0]:
%cd ./gdrive/My\ Drive/smm4h

/content/gdrive/My Drive/smm4h


In [0]:
! git clone

Initialized empty Git repository in /content/gdrive/My Drive/smm4h/.git/


хэлоу

In [0]:
import pandas as pd
import numpy as np
import re
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score
from tqdm import tqdm_notebook
from sklearn.feature_extraction.text import TfidfVectorizer


DATA_FOLDER = './gdrive/My Drive/' + 'data_smm4h/'
data_train = pd.read_csv(DATA_FOLDER + 'task2_ru_training.tsv', sep='\t')
data_val = pd.read_csv(DATA_FOLDER + 'task2_ru_validation.tsv', sep='\t')

Train = data_train[['tweet', 'class']]
Train.fillna('', inplace=True)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4259: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  **kwargs


In [0]:
Train['class'].value_counts()

0    5557
1     533
Name: class, dtype: int64

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pickle

tf_idf = TfidfVectorizer()
tf_idf.fit(Train['tweet'])

# with open("tfidf.pkl", 'wb') as handle:
#     pickle.dump(tf_idf, handle)

X_train_tf_idf = tf_idf.transform(Train['tweet'])
y_train = Train['class']

In [0]:
Val = data_val[['tweet', 'class']]
Val.fillna('', inplace=True)

X_val = Val[['tweet']]
X_val_tf_idf = tf_idf.transform(X_val['tweet'])
y_val = Val['class']

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4259: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  **kwargs


---------------------

In [0]:
from sklearn.metrics import roc_auc_score

def eval_model(X, X_prepared, y_true, model, verbose=True):
    y_prob = model.predict_proba(X_prepared)[:,1]
    y_prob = pd.Series(data=y_prob, index=y_true.index)
    
    av_auc = 0
    for cat in X.category.unique():
        if len(y_true[indexes].unique()) == 1:
            continue
        cat_auc = roc_auc_score(y_true[indexes], y_prob[indexes])
        av_auc += cat_auc
        if verbose:
            print((cat + "(" + str(len(y_true[indexes])) + ") :").ljust(40), cat_auc.round(5))
    av_auc /= len(X.category.unique())
    if verbose:
        print()
        print("average roc_auc by all categories:", av_auc.round(5))
    return av_auc

## LightGBM

In [0]:
%%time

from lightgbm import LGBMClassifier

lgbmodel = LGBMClassifier(
    boosting_type="gbdt",
    objective="binary",
    # num_leaves=30,
    # max_depth=10,
    # n_estimators=800,
#     min_child_samples=100,
#     max_bin=100,
#     subsample=0.7,
#     subsample_freq=1,
#     colsample_bytree=0.9,
#     min_child_weight=0,
#     scale_pos_weight=200
)

lgbmodel = lgbmodel.fit(X_train_tf_idf, 
                        y_train, 
#                         eval_set=[(X_train_tf_idf, y_train),(X_test_tf_idf, y_test)],
                        eval_metric='auc'
                        )

CPU times: user 2.35 s, sys: 26.8 ms, total: 2.37 s
Wall time: 1.26 s


In [0]:
y_prob = lgbmodel.predict_proba(X_val_tf_idf)[:,1]
y_pred = lgbmodel.predict(X_val_tf_idf)
f1_score(y_val, y_pred)

0.29545454545454547

# LogReg

In [0]:
from sklearn.linear_model import LogisticRegression

log_regr = LogisticRegression()
log_regr.fit(X_train_tf_idf, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [0]:
y_prob = log_regr.predict_proba(X_val_tf_idf)[:,1]
y_pred = log_regr.predict(X_val_tf_idf)
print(roc_auc_score(y_val, y_prob))
print(f1_score(y_val, y_pred))

0.8291787784796766
0.07092198581560283


In [0]:
len(y_val[y_val == 1])

133

# LinearSVC

In [0]:
from sklearn.svm import LinearSVC

svc = LinearSVC(random_state=0)

svc.fit(X_train_tf_idf, y_train)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=0, tol=0.0001,
          verbose=0)

In [0]:
y_pred = svc.predict(X_val_tf_idf)
f1_score(y_val, y_pred)

0.28415300546448086

# SVC

In [0]:
from sklearn.linear_model import SGDClassifier

clf = SGDClassifier(loss='hinge',
                    # learning_rate='optimal', # 'adaptive', 'invscaling', 'optimal'
                    )
clf.fit(X_train_tf_idf, y_train)

y_pred = clf.predict(X_val_tf_idf)
f1_score(y_val, y_pred)

0.2934782608695652

# CatBoost

In [0]:
! pip install catboost

     |████████████████████████████████| 64.4MB 49kB/s 


In [0]:
from catboost import CatBoostClassifier, Pool, cv

cat = CatBoostClassifier(
    # custom_loss=['Accuracy'],
    # random_seed=42,
    # logging_level='Silent',
    # iterations=1000,
    # max_depth=4,
    task_type='GPU'
)


cat.fit(
    X_train_tf_idf, y_train
);

219:	learn: 0.2379804	total: 17.2s	remaining: 1m 1s
220:	learn: 0.2378115	total: 17.3s	remaining: 1m
221:	learn: 0.2377091	total: 17.4s	remaining: 1m
222:	learn: 0.2374876	total: 17.4s	remaining: 1m
223:	learn: 0.2373277	total: 17.5s	remaining: 1m
224:	learn: 0.2372140	total: 17.6s	remaining: 1m
225:	learn: 0.2370706	total: 17.7s	remaining: 1m
226:	learn: 0.2369141	total: 17.8s	remaining: 1m
227:	learn: 0.2368037	total: 17.8s	remaining: 1m
228:	learn: 0.2365095	total: 17.9s	remaining: 1m
229:	learn: 0.2364118	total: 18s	remaining: 1m
230:	learn: 0.2362266	total: 18.1s	remaining: 1m
231:	learn: 0.2360056	total: 18.1s	remaining: 1m
232:	learn: 0.2358587	total: 18.2s	remaining: 59.9s
233:	learn: 0.2356680	total: 18.3s	remaining: 59.9s
234:	learn: 0.2355132	total: 18.4s	remaining: 59.8s
235:	learn: 0.2353495	total: 18.4s	remaining: 59.7s
236:	learn: 0.2351887	total: 18.5s	remaining: 59.6s
237:	learn: 0.2350869	total: 18.6s	remaining: 59.5s
238:	learn: 0.2349356	total: 18.7s	remaining: 59.4

In [0]:
y_prob = cat.predict_proba(X_val_tf_idf)[:,1]
y_pred = cat.predict(X_val_tf_idf)
f1_score(y_val, y_pred)

0.1871345029239766

------------------------